In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

import psycopg2 as pg
import pandas.io.sql as pd_sql

from sklearn.model_selection import train_test_split

In [2]:
connection_args = {
    'host': '18.191.95.148',
    'user': 'ubuntu',
    'dbname': 'mcnulty',
    'port': 5432
}

connection = pg.connect(**connection_args)

In [3]:
query = '''with data_mat_clean as (SELECT *
    FROM data_mat
    WHERE milex != -9 AND milper != -9 AND irst != -9 AND pec != -9 AND tpop != -9
    AND upop != -9 AND cinc != -9),

data_trade_clean as (SELECT *
    FROM data_trade
    WHERE imports IS NOT NULL AND exports IS NOT NULL),

data_d1_recent as(SELECT *
    FROM data_d1
    WHERE orig = 1 AND styear >= 1914 AND hostlev != 0),
    
disp_mat as (SELECT a.dispnum3, a.ccode, a.stabb, a.styear, a.hostlev, a.orig,
    b.milex, b.milper, b.irst, b.pec, b.tpop, b.upop, b.cinc
    FROM data_d1_recent a
    INNER JOIN data_mat_clean b
    ON a.ccode = b.ccode AND a.styear = b.year)
    
SELECT c.dispnum3, c.ccode, c.stabb, c.styear, c.hostlev, c.orig,
    c.milex, c.milper, c.irst, c.pec, c.tpop, c.upop, c.cinc,
    d.imports, d.exports
    FROM disp_mat c
    INNER JOIN data_trade_clean d
    ON c.ccode = d.ccode AND c.styear = d.year
;'''

disp_mat_trade = pd_sql.read_sql(query, connection)

In [4]:
disp_mat_trade.head(15)

,dispnum3,ccode,stabb,styear,hostlev,orig,milex,milper,irst,pec,tpop,upop,cinc,imports,exports
0,100,2,USA,1914,4,1,253205,166,23890,556778,99111.0,23340.0,0.206364,1924.0,2420.0
1,69,2,USA,1914,4,1,253205,166,23890,556778,99111.0,23340.0,0.206364,1924.0,2420.0
2,398,2,USA,1915,2,1,257648,174,32667,580731,100546.0,24008.0,0.222083,1703.0,2820.0
3,1775,2,USA,1915,2,1,257648,174,32667,580731,100546.0,24008.0,0.222083,1703.0,2820.0
4,399,2,USA,1916,5,1,277421,179,43460,646120,101961.0,24688.0,0.232936,2424.0,5554.0
5,321,2,USA,1916,4,1,277421,179,43460,646120,101961.0,24688.0,0.232936,2424.0,5554.0
6,2733,2,USA,1916,1,1,277421,179,43460,646120,101961.0,24688.0,0.232936,2424.0,5554.0
7,322,2,USA,1916,4,1,277421,179,43460,646120,101961.0,24688.0,0.232936,2424.0,5554.0
8,1660,2,USA,1917,4,1,658584,644,45784,710959,103268.0,25310.0,0.244028,3005.0,6318.0
9,2184,2,USA,1918,4,1,7014226,2897,45176,743925,103208.0,26020.0,0.294872,3105.0,6402.0


In [5]:
disp_mat_trade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3614 entries, 0 to 3613
Data columns (total 15 columns):
dispnum3    3614 non-null int64
ccode       3614 non-null int64
stabb       3614 non-null object
styear      3614 non-null int64
hostlev     3614 non-null int64
orig        3614 non-null int64
milex       3614 non-null int64
milper      3614 non-null int64
irst        3614 non-null int64
pec         3614 non-null int64
tpop        3614 non-null float64
upop        3614 non-null float64
cinc        3614 non-null float64
imports     3614 non-null float64
exports     3614 non-null float64
dtypes: float64(5), int64(9), object(1)
memory usage: 423.6+ KB


In [6]:
with open("data/pickle_files/peace_originators_clean.pickle", "wb") as f:
    pickle.dump(disp_mat_trade, f)

In [7]:
X = disp_mat_trade.drop(['hostlev', 'dispnum3', 'ccode', 'stabb', 'styear', 'orig'], axis=1)
y = disp_mat_trade['hostlev']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [9]:
pickle_object = (X_train, X_test, y_train, y_test)
with open("data/pickle_files/peace_originators_split.pickle", "wb") as f:
    pickle.dump(pickle_object, f)